In [1]:
#!pip install pyedflib
#!pip install PyWavelets

import pyedflib
import numpy as np
import pandas as pd
import matplotlib
import pywt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from numpy import *
from scipy.signal import *
from numpy.fft import * 
from matplotlib import *
from scipy import *
from pylab import *
import pandas
from scipy.fftpack import fft,ifft
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy import fftpack
import math
import pyedflib
import pyeeg
import pywt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from pylab import *

import numpy as np
from numpy import *
from numpy.fft import * 
import scipy
from scipy.signal import *
from scipy import *
from scipy.fftpack import fft,ifft
import scipy.signal as signal
from scipy import fftpack

import io
import struct
import pickle

In [2]:
def read_pickle(file_dir):   
    pickle_L=[]   
    pickle_name=[]
    for dirpath, dirnames, filenames in os.walk(file_dir):  
        for file in filenames :  
            if os.path.splitext(file)[1] == '.edf':  
                pickle_L.append(os.path.join(dirpath, file))  
                pickle_name.append(os.path.join(file))  
            
    return pickle_L 
    #return  pickle_name
    #return name
    

In [3]:
import os  
import pandas as pd
def file_name(file_dir):   
    L=[]   
    name=[]
    for dirpath, dirnames, filenames in os.walk(file_dir):  
        for file in filenames :  
            if os.path.splitext(file)[1] =='.edf':  
                L.append(os.path.join(dirpath, file))  
                name.append(os.path.splitext(file)[0])  
            
    #return L 
    return name
    #return name

In [4]:
import os  
import pandas as pd
def edf_file(file_dir):   
    L=[]   
    R=[]
    for dirpath, dirnames, filenames in os.walk(file_dir):  
        for file in filenames :  
            if os.path.splitext(file)[1] == '.edf':  
                L.append(os.path.join(dirpath, file))  
                R.append(os.path.join(file))
                #name.append(os.path.splitext(file)[0])  
            
    #return L 
    return R
    #return name

In [ ]:
p="file_path\\"

In [6]:
la = "file_path\\"

In [7]:
pickle_fil=read_pickle(p)

In [9]:
fnname=edf_file(p)

In [10]:
new=file_name(p)

In [11]:
na=[]
for i in range(0,len(new)):
    na.append(new[i])

## Sub-Frequency

In [13]:
from scipy.signal import hilbert, chirp
from scipy.signal import butter, lfilter
import pyeeg

def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

epoch=int(0.5*250)

# overlap window

In [14]:
fs=500
epoch=int(2.5*fs)

In [15]:
def window_stack(a, stepsize=1, width=2):
    n = a.shape[0]
    return np.hstack( a[i:1+n+i-width:stepsize] for i in range(0,width) )

In [ ]:
for i in range (0, len(pickle_fil)):
    edf_file=read_pickle(p)[i]
    signals, signal_headers, header =pyedflib.highlevel.read_edf(edf_file)
    f = pyedflib.EdfReader(edf_file)
    print(f)
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    new_signal_label=[]
    sigbufs = np.zeros((n, f.getNSamples()[0]))

    for p in np.arange(2):
        sigbufs[p, :] = f.readSignal(p)
    df_signals = pd.DataFrame(sigbufs)
    df_signals = df_signals.transpose()
    #df_signals.columns = signal_labels
    df_signals.columns=['EEG1','EEG2']
    
    #df = df.transpose()
    #df.columns = signal_labels
    #fs1=f.getSampleFrequency(3)
    #print(fs1)
   # res = signal.resample(df,int(len(df)/fs1*250))
    #df_signals =pd.DataFrame(res)
    #df_signals.columns = new_signal_label
    
    df_signals['Annotation']=0    
    manual=df_signals['Annotation']
    
    df_signals=df_signals['EEG2']-mean(df_signals['EEG2'])
    df_signals=pd.DataFrame(df_signals)
    df_signals.columns=['EEG2']
    
    
    
    # Notch filter 
    f0 = 50.0  # Frequency to be removed from signal (Hz)
    #Q = 15.0  # Quality factor
    Q=35
    # Design notch filter
    b, a = signal.iirnotch(f0, Q,fs)

    freq, h = signal.freqz(b, a)

    n = len(df_signals)                  # length of the signal
    k = np.arange(n)
    T = n/fs
    frq = k/T                   # two sides frequency range
    frq1 = frq[range(int(n/2))] # one side frequency range


    df_EEG2=pd.DataFrame(signal.filtfilt(b, a, df_signals['EEG2'], method='gust'))

    df_EEG2_notch=df_EEG2[0]

    # mean 
    R_reshape=pd.DataFrame(np.array(df_EEG2_notch).reshape(int(n/epoch),-1))
    R_overlap=pd.DataFrame(window_stack(R_reshape, stepsize=1, width=2))
    R_t=R_overlap.T

    mean_EEG2=R_t.mean()

    # TKEO
    signal_EEG2 = df_EEG2_notch.to_numpy()

    # [Application of TKEO Operator]
    tkeo_EEG2 = []


    for iii in range(0, len(signal_EEG2)):
        if iii == 0 or iii== (len(signal_EEG2 ) - 1):
            tkeo_EEG2.append(signal_EEG2[iii])

        else:
            tkeo_EEG2.append((signal_EEG2[iii]** 2) - (signal_EEG2[iii + 1] * signal_EEG2[iii - 1]))

    EEG2_tkeo=pd.DataFrame(np.array(tkeo_EEG2).reshape(int(n/epoch),-1))
    EEG2_tkeo_overlap=pd.DataFrame(window_stack(EEG2_tkeo, stepsize=1, width=2))
    TKE_EEG2=EEG2_tkeo_overlap.T
    TKEO_EEG2=mean(TKE_EEG2)


    # Frequency
    EEG2_P_delta=(butter_bandpass_filter(mean_EEG2, 0.1, 4, fs, order=6))**2
    EEG2_P_theta=(butter_bandpass_filter(mean_EEG2, 4, 8, fs, order=6))**2
    EEG2_P_alpha=(butter_bandpass_filter(mean_EEG2, 8, 16, fs, order=6))**2
    EEG2_P_beta=(butter_bandpass_filter(mean_EEG2, 16, 32, fs, order=6))**2
    EEG2_P_gamma=(butter_bandpass_filter(mean_EEG2, 32, 64, fs, order=6))**2

    EEG2_P_total=mean_EEG2**2
    EEG2_rel_delta=EEG2_P_delta/EEG2_P_total
    EEG2_rel_theta=EEG2_P_theta/EEG2_P_total
    EEG2_rel_alpha=EEG2_P_alpha/EEG2_P_total
    EEG2_rel_beta=EEG2_P_beta/EEG2_P_total;
    EEG2_rel_gamma=EEG2_P_gamma/EEG2_P_total;


    EEG2_P_delta=pd.DataFrame(EEG2_P_delta)
    EEG2_P_theta=pd.DataFrame(EEG2_P_theta)
    EEG2_P_alpha=pd.DataFrame(EEG2_P_alpha)
    EEG2_P_beta=pd.DataFrame(EEG2_P_beta)
    EEG2_P_gamma=pd.DataFrame(EEG2_P_gamma)
    EEG2_P_total=pd.DataFrame(EEG2_P_total)
    EEG2_rel_delta=pd.DataFrame(EEG2_rel_delta)
    EEG2_rel_theta=pd.DataFrame(EEG2_rel_theta)
    EEG2_rel_alpha=pd.DataFrame(EEG2_rel_alpha)
    EEG2_rel_beta=pd.DataFrame(EEG2_rel_beta)
    EEG2_rel_gamma=pd.DataFrame(EEG2_rel_gamma)


    analytic_EEG2 = hilbert(mean_EEG2)
    EEG2_envelope = np.abs(analytic_EEG2)
    EEG2_envelope = pd.DataFrame(EEG2_envelope)



    R1=[]
    p1=[]
    skew1=[]
    kur1=[]
    var1=[]

    for i1 in range(0,R_t.shape[1]):
        R1.append(pyeeg.hjorth(R_t[i1].values.tolist(), D=None))
        p1.append(pyeeg.pfd(R_t[i1].values.tolist()))
        ### SKEWNESS
        skew1.append(scipy.stats.skew(R_t[i1].values.tolist(), axis=0, bias=True))
        ### Kurtosis
        kur1.append(scipy.stats.kurtosis(R_t[i1].values.tolist()))
        ### VARIANCE
        var1.append(numpy.var(R_t[i1].values.tolist(), axis=None, dtype=None, out=None, ddof=0))



    hjR1=pd.DataFrame(np.asarray(R1))
    EEG2_mobility=pd.DataFrame(hjR1[0])
    #R_hurst=pd.DataFrame(np.asarray(h1))
    EEG2_pfd=pd.DataFrame(np.asarray(p1))
    #R_hfd=pd.DataFrame(np.asarray(hf1))
    EEG2_skew=pd.DataFrame(np.asarray(skew1))
    EEG2_kurtosis=pd.DataFrame(np.asarray(kur1))
    EEG2_var=pd.DataFrame(np.asarray(var1))  

    # target
    label=pd.read_excel('label/'+str(na[i])+'.xlsx')

    label[['S_hour','S_minute','S_second']]=label['start'].str.split('.',expand=True)
    label[['E_hour','E_minute','E_second']]=label['end'].str.split('.',expand=True)

    label['S_hour']=label['S_hour'].astype(int)
    label['S_minute']=label['S_minute'].astype(int)
    label['S_second']=label['S_second'].astype(int)
    label['E_hour']=label['E_hour'].astype(int)
    label['E_minute']=label['E_minute'].astype(int)
    label['E_second']=label['E_second'].astype(int)

    label['Start_signal']=(label['S_hour']*3600+label['S_minute']*60+label['S_second'])*500
    label['End_signal']=(label['E_hour']*3600+label['E_minute']*60+label['E_second'])*500
    label['Duration']=label['End_signal']-label['Start_signal']

    for ppppp in range(0,len(label)):
        manual[label.Start_signal.values[ppppp]:label.End_signal.values[ppppp]]=1

    mark_reshape=pd.DataFrame(np.array(manual).reshape(int(n/epoch),-1))
    mark_overlape=pd.DataFrame(window_stack(mark_reshape, stepsize=1, width=2))
    mark_tar=mark_overlape.T    

    sp_tar=pd.DataFrame(sum(mark_tar))
    sp_tar.columns=['seizure']

    sp_tar.loc[sp_tar['seizure']>=epoch, 'target'] = 1
    sp_tar.loc[sp_tar['seizure']<epoch, 'target'] = 0
    sp_tar['target'] = sp_tar['target'].astype('int')

    target=sp_tar['target']
    
    df_combine = pd.concat([target,mean_EEG2, EEG2_mobility, TKEO_EEG2, EEG2_P_delta,EEG2_P_theta,
                            EEG2_P_alpha,EEG2_P_beta,EEG2_P_gamma,EEG2_P_total,EEG2_rel_delta,EEG2_rel_theta,
                            EEG2_rel_alpha,EEG2_rel_beta,EEG2_rel_gamma,EEG2_pfd,EEG2_skew,
                            EEG2_kurtosis,EEG2_var,EEG2_envelope], axis=1)
    #data
    df_combine.columns = ['target','mean_EEG2', 'EEG2_mobility', 'TKEO_EEG2', 'EEG2_P_delta','EEG2_P_theta',
                                'EEG2_P_alpha','EEG2_P_beta','EEG2_P_gamma','EEG2_P_total','EEG2_rel_delta','EEG2_rel_theta',
                                'EEG2_rel_alpha','EEG2_rel_beta','EEG2_rel_gamma','EEG2_pfd','EEG2_skew',
                                'EEG2_kurtosis','EEG2_var','EEG2_envelope']

    df1=open("filename_"+str(na[i])+'.pickle','wb')# 注意一定要写明是wb 而不是w.
    #最关键的是这步，将内容装入打开的文件之中（内容，目标文件）
    pickle.dump(df_combine,df1)
    df1.close()